In [2]:
import pandas as pd
import numpy as np
from path_prefix import PATH_PREFIX

In [4]:
df = pd.read_csv(f'{PATH_PREFIX}/data/osstate/extracted/states_2022-04-04-00.csv.gz', compression='gzip')
# Drop all rows with NaN values
df = df.dropna()
df.head()

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact
0,1649030400,7c02dc,-37.662643,145.395053,50.857154,38.840837,5.20192,AUM,False,False,False,4001.0,1615.44,1676.40,1.649030e+09,1.649030e+09
5,1649030400,4ba8d0,49.376936,11.491334,236.586670,121.755981,0.00000,THY1DU,False,False,False,1176.0,11277.60,10927.08,1.649030e+09,1.649030e+09
8,1649030400,7c7f14,-37.217331,145.824455,93.220887,37.375807,0.00000,ZDY,False,False,False,3236.0,2727.96,2781.30,1.649030e+09,1.649030e+09
10,1649030400,8963ce,40.657608,31.010986,298.288810,110.810486,-0.32512,ETD69K,False,False,False,4716.0,11887.20,11879.58,1.649030e+09,1.649030e+09
17,1649030400,7c6b2f,-37.812614,145.168563,170.978929,332.201314,-4.55168,JST702,False,False,False,3663.0,3314.70,3421.38,1.649030e+09,1.649030e+09


In [5]:
# Filter out all rows with geoaltitude between 300 and 600 feet
df_groundprox = df[(df['geoaltitude'] > 300) & (df['geoaltitude'] < 600)]
df_groundprox.head()

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact
57,1649030410,aacd0a,40.933429,-95.902280,46.528040,35.095817,1.30048,N79501,False,False,False,1200.0,609.60,571.50,1.649030e+09,1.649030e+09
117,1649030410,a70ec4,41.878952,-87.910873,58.214061,273.039400,-0.65024,N554F,False,False,False,1200.0,335.28,403.86,1.649030e+09,1.649030e+09
185,1649030410,a8b8b9,41.965622,-87.962099,67.906608,90.000000,-3.57632,ENY3817,False,False,False,5623.0,327.66,320.04,1.649030e+09,1.649030e+09
261,1649030410,ab7376,40.385277,-73.868013,99.702645,306.396878,-0.32512,DAL371,False,False,False,1356.0,647.70,563.88,1.649030e+09,1.649030e+09
277,1649030410,a33241,37.675690,-122.107310,98.711603,102.949854,0.65024,N305DS,False,False,False,1200.0,457.20,426.72,1.649030e+09,1.649030e+09


In [6]:
# Add an ident column to the dataframe, which is the concatenation of the icao24 and callsign columns
df_groundprox['ident'] = (df_groundprox['callsign'].str.strip()+'_'+df_groundprox['icao24'].str.strip())
df_groundprox.head()

/var/folders/8k/_h2zg_z51q9fpsbmsh42bd3h0000gn/T/ipykernel_21127/3006671645.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_groundprox['ident'] = (df_groundprox['callsign'].str.strip()+'_'+df_groundprox['icao24'].str.strip())


,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,ident
57,1649030410,aacd0a,40.933429,-95.902280,46.528040,35.095817,1.30048,N79501,False,False,False,1200.0,609.60,571.50,1.649030e+09,1.649030e+09,N79501_aacd0a
117,1649030410,a70ec4,41.878952,-87.910873,58.214061,273.039400,-0.65024,N554F,False,False,False,1200.0,335.28,403.86,1.649030e+09,1.649030e+09,N554F_a70ec4
185,1649030410,a8b8b9,41.965622,-87.962099,67.906608,90.000000,-3.57632,ENY3817,False,False,False,5623.0,327.66,320.04,1.649030e+09,1.649030e+09,ENY3817_a8b8b9
261,1649030410,ab7376,40.385277,-73.868013,99.702645,306.396878,-0.32512,DAL371,False,False,False,1356.0,647.70,563.88,1.649030e+09,1.649030e+09,DAL371_ab7376
277,1649030410,a33241,37.675690,-122.107310,98.711603,102.949854,0.65024,N305DS,False,False,False,1200.0,457.20,426.72,1.649030e+09,1.649030e+09,N305DS_a33241


In [7]:
unique_idents = df_groundprox['ident'].unique()
print(f'Number of unique idents: {len(unique_idents)}')

Number of unique idents: 2632


In [10]:
# A flight is considered to be taking off if final altitude is greater than initial altitude
n_takeoffs = 0
n_landings = 0

for ident in unique_idents:
    df_ident = df_groundprox[df_groundprox['ident'] == ident]
    df_ident = df_ident.sort_values(by='lastposupdate')
    initial_altitude = df_ident['geoaltitude'].iloc[0]
    final_altitude = df_ident['geoaltitude'].iloc[-1]
    if final_altitude > initial_altitude:
        # Flight is taking off, we note the time of takeoff and the position where the flight took off
        time_takeoff = df_ident['lastposupdate'].iloc[0]
        time_landing = -1
        lat_prox = df_ident['lat'].iloc[0]
        lon_prox = df_ident['lon'].iloc[0]
        n_takeoffs += 1
    else:
        time_landing = df_ident['lastposupdate'].iloc[-1]
        time_takeoff = -1
        lat_prox = df_ident['lat'].iloc[-1]
        lon_prox = df_ident['lon'].iloc[-1]
        n_landings += 1

print(f'Number of takeoffs: {n_takeoffs}')
print(f'Number of landings: {n_landings}')
        

Number of takeoffs: 692
Number of landings: 1940


# Misc

In [6]:
# For fun code: count the number of total flights
import os 

extracted_files = os.listdir(f'{PATH_PREFIX}/data/osstate/extracted')
extracted_files = [file for file in extracted_files if file.endswith('.csv.gz')]
print(f'There are {len(extracted_files)} files in the extracted directory')

n_flights = 0
for file in extracted_files:
    df = pd.read_csv(f'{PATH_PREFIX}/data/osstate/extracted/{file}', compression='gzip')
    df = df.dropna()
    n_flights += len(df['callsign'].unique())
    print(f'                                              ', end='\r')
    print(f'Total flights so far: {n_flights}', end='\r')

print(f'Total number of flights: {n_flights}')



There are 312 files in the extracted directory
Total number of flights: 1760605              


In [4]:
# Load the airport data
df_airports = pd.read_csv(f'{PATH_PREFIX}/data/airport-codes.csv')
print(f'Number of airports: {len(df_airports)}')
# Only keep medium and large airports
df_airports = df_airports[df_airports['type'].isin(['medium_airport', 'large_airport'])]
print(f'Number of medium and large airports: {len(df_airports)}')
# The coordinates column is a string with the format "latitude, longitude", we split it into two columns and convert them to float
df_airports[['latitude', 'longitude']] = df_airports['coordinates'].str.split(', ', expand=True)
# Drop the coordinates column
df_airports = df_airports.drop(columns=['coordinates'])
# Save the airport data
df_airports.to_csv(f'{PATH_PREFIX}/data/airport-codes-xl.csv', index=False)

Number of airports: 75848
Number of medium and large airports: 5216
